# `CLOUD.jl` - 1D linear advection-diffusion example

In [1]:
using CLOUD, OrdinaryDiffEq, Plots

LoadError: Failed to precompile CLOUD [fb992021-99c7-4c2d-a14b-5e48ac4045b2] to /Users/tristanmontoya/.julia/compiled/v1.7/CLOUD/jl_FKMQhj.

Define the physical problem

In [2]:
a = 1.0  # wave speed
b = 5.0e-2 # diffusion coefficient
L = 1.0  # domain length
T = 1.0  # end time for one period

initial_data = InitialDataSine(1.0,2π)
conservation_law = LinearAdvectionDiffusionEquation(a,b)
exact_solution = ExactSolution(conservation_law,initial_data);

LoadError: UndefVarError: InitialDataSine not defined

Set discretization parameters

In [3]:
M = 4
p = 5
form = WeakConservationForm()
strategy = Lazy()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.05*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*100));

LoadError: UndefVarError: WeakConservationForm not defined

Set up a `DGSEM` scheme with LGL quadrature of degree `p` with `M` elements

In [4]:
reference_approximation =ReferenceApproximation(
    DGSEM(p), Line(), volume_quadrature_rule=LGLQuadrature())

mesh = uniform_periodic_mesh(
    reference_approximation.reference_element, (0.0,L), M)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_diffusion_1d_dgsem/", overwrite=true, clear=true);

LoadError: UndefVarError: DGSEM not defined

Run the solver and print timer outputs

In [5]:
ode_problem = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy)
save_solution(ode_problem.u0, 0.0, results_path, 0)
CLOUD_reset_timer()
sol = solve(ode_problem, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

LoadError: UndefVarError: semidiscretize not defined

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [6]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), exact_solution, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

LoadError: UndefVarError: ErrorAnalysis not defined

Plot the exact and numerical solutions

In [7]:
plot(spatial_discretization, last(sol.u), 
    exact_solution=exact_solution, time=T, ylims=[-1.0,1.0])

LoadError: UndefVarError: sol not defined

Animate the numerical solution

In [8]:
using Printf
anim = @animate for i ∈ eachindex(sol.u)
    plot(spatial_discretization, sol.u[i], 
        ylims=[-1.0,1.0], ylabel="\$U^h(x,t)\$",
        label=string("t = ", @sprintf "%.2f" sol.t[i]))
end
gif(anim, "advection_diffusion_solution.gif", fps = 10)

LoadError: LoadError: UndefVarError: @animate not defined
in expression starting at In[8]:2